In [69]:
import os

In [70]:
%pwd

'g:\\'

In [71]:
os.chdir("../")

In [72]:
%pwd

'g:\\'

In [73]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [74]:
from CNN_Classifier.constants import *
from CNN_Classifier.utils.common import read_yaml, create_directories

In [75]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [76]:
import os
import sys
import urllib.request as request
import zipfile
from CNN_Classifier.logging import logging
from CNN_Classifier.exception_handler import CustomException
from CNN_Classifier.utils.common import get_size

In [77]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logging.info(f"{filename} download! with following info: \n{headers}")
        else:
            logging.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [79]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise CustomException(e,sys)

[2023-06-06 12:51:38,662: INFO: __init__: Error message completed]
[2023-06-06 12:51:38,664: INFO: __init__: Error message completed]


CustomException: Error occured in python script name [C:\Users\user\AppData\Local\Temp\ipykernel_26552\1954843609.py] line number [2] error message [Error occured in python script name [g:\end-to-end dl project\src\CNN_Classifier\utils\common.py] line number [18] error message [[Errno 2] No such file or directory: 'config\\config.yaml']]